## Задание

Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  

Установка необходимых библиотек:

In [15]:
!pip install gensim --upgrade

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [8]:
!pip install natasha

     |████████████████████████████████| 34.4MB 112kB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 8.2MB 6.1MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26100 sha256=743b6330fa9f9411133a71c5eeb3db8118a17ff065758f3486c7f7050a754ac2
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


Скачиваем с **rusvectores** модель araneum_none_fasttextcbow_300_5_2018 (fasttext) и разархивируем ее

In [ ]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-10-21 00:00:35--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  11.3MB/s    in 3m 55s  

2020-10-21 00:04:32 (10.9 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [ ]:
!tar -xf araneum_none_fasttextcbow_300_5_2018.tgz

Импорт библиотек:

In [60]:
import re
import nltk
import pymorphy2
import pandas as pd
import numpy as np
from math import log
from razdel import tokenize
from string import punctuation
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from natasha import NewsNERTagger, NewsEmbedding, Doc, Segmenter

Загружаем модель fasttext

In [2]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

Открываем данные с вопросами по COVID-19

In [5]:
answers_df = pd.read_excel('answers_base.xlsx')
queries_df = pd.read_excel('queries_base.xlsx')
#в датафрейме с запросами есть пропуски, убираем их
queries_df = queries_df[['Текст вопроса', 'Номер связки\n']].dropna()

Делим выборку на тренировочную (все answers и 70% queries) и тестовую (30% queries)

In [7]:
queries_train, queries_test = train_test_split(queries_df, test_size=0.3, random_state=0)
documents = answers_df['Текст вопросов'].append(queries_train['Текст вопроса'], ignore_index=True)
answers_train = answers_df['Номер связки'].append(queries_train['Номер связки\n'], ignore_index=True)
answers_test = queries_test['Номер связки\n']

Загружаем необходимые модули для предобработки текста

In [63]:
nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()
stopwords = set(stopwords.words('russian'))
punkt = punctuation + '«»—…“”*№–'

#сущности из natasha
emb = NewsEmbedding()
segmenter = Segmenter()
ner_tagger = NewsNERTagger(emb)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
def tokenizing(text):
  """Функция, делящая текст на токены"""
  tokens = list(tokenize(text))
  return [_.text.lower() for _ in tokens]

def preprocessing(text):
  """Основная предобработка: удаление стоп-слов, запятых и лемматизация"""
    tokens = tokenizing(text)
    clean_text = [x for x in tokens if x not in stopwords and x not in punkt]
    clean_text = [morph.parse(word)[0].normal_form for word in clean_text if re.match('\W+', word) is None]
    return clean_text

def preprocess_with_natasha(text: str) -> str:
  """Удаление именованных сущностей """
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    new_text = text
    for entity in doc.spans:
        new_text = new_text.replace(text[entity.start:entity.stop], '')
    return new_text

Я выбрала именно наташу, потому что в прошлой домашке она показала себя лучше чем deeppavlov!

Применим функции предобработки к текстам:

In [65]:
prep_documents = documents.apply(preprocessing)
test_queries = queries_test['Текст вопроса'].apply(preprocessing)

In [66]:
ner_documents = documents.apply(lambda x: preprocessing(preprocess_with_natasha(x)))
ner_test_queries = queries_test['Текст вопроса'].apply(lambda x: preprocessing(preprocess_with_natasha(x)))

## 1. Преобразование каждого документа в вектор через усреднение

In [30]:
def normalize_vec(v):
  """ Функция для нормализации вектора"""
     return v / np.sqrt(np.sum(v ** 2))


def text_to_vec(document, dim=model.vector_size):
  """ Функция, переводящая текст документа в вектор через усреднение векторов слов"""
    all_embs = []
    for word in document:
      # если слово в модели, добавляем в список его вектор
      if word in model:
        all_embs.append(normalize_vec(model[word]))
      #если ни одного слова из документа нет в модели, возвращаем нули
    if len(all_embs) == 0:
      return np.zeros(dim)
    return np.mean(all_embs,axis=0)

Векторизуем каждый документ и запрос коллекции:

In [67]:
vec_documents = prep_documents.apply(text_to_vec)
vec_queries = test_queries.apply(text_to_vec)

In [125]:
vec_documents_ner = ner_documents.apply(text_to_vec)
vec_queries_ner = ner_test_queries.apply(text_to_vec)

Мы получили список, каждый элемент которого - numpy array, а хочется матрицу. Поэтому применим функцию numpy.vstack

In [126]:
vec_documents = np.vstack(vec_documents)
vec_queries = np.vstack(vec_queries)

vec_documents_ner = np.vstack(vec_documents_ner)
vec_queries_ner = np.vstack(vec_queries)

Приступаем к ранжированию:

In [127]:
def ranging(query, vectors):
  """Функция, возвращающая топ1 ответ запросу query по косинусной близости векторов"""
  sims = enumerate(vectors.dot(query))
  sorted_docs = sorted(sims, key=lambda x: x[1], reverse=True)
  top1_doc = sorted_docs[0][0]
  return answers_train[top1_doc]

Для оценивания результатов напишем функцию, считающую долю верно угаданных ответов

In [128]:
def accuracy(predicted, answers_test=answers_test):
  """Функция, которая считает метрику accuracy для полученных ответов"""
  right_answers = answers_test[answers_test==predicted].shape[0]
  return right_answers/answers_test.shape[0]

Найдем топ1 нужный ответ для каждого запроса в коллекции

In [129]:
predicted = [ranging(q, vec_documents) for q in vec_queries]
predicted_ner = [ranging(q, vec_documents_ner) for q in vec_queries_ner]

Померяем качество:

In [130]:
print('Качество поиска через умножение матрицы на вектор: ', accuracy(predicted))
print('Качество поиска через умножение матрицы на вектор с удалением NER', accuracy(predicted_ner))

Качество поиска через умножение матрицы на вектор:  0.16739446870451238
Качество поиска через умножение матрицы на вектор с удалением NER 0.19068413391557495


## 2. Эксперимент - представление документа в виде матрицы


У нас уже есть предобработанные тексты, и мы сразу их будем представлять в другом виде:

In [135]:
def create_doc_matrix(text, dim=model.vector_size):
  """Функция с семинара, создающая из документа матрицу векторов его слов"""
    lemmas_vectors = np.zeros((len(text), dim))
    vec = np.zeros((dim,))

    for idx, lemma in enumerate(text):
        if lemma in model:
            lemmas_vectors[idx] = normalize_vec(model[lemma])
            
    return lemmas_vectors  

Применяем функцию выше ко всем запросам и документам

In [139]:
matrix_docs = prep_documents.apply(create_doc_matrix)
matrix_queries = test_queries.apply(create_doc_matrix)

matrix_docs_ner = ner_documents.apply(create_doc_matrix)
matrix_queries_ner = ner_test_queries.apply(create_doc_matrix)

In [140]:
def search(docs, query, reduce_func=np.max, axis=0):
  """Функция, возвращающая топ1 ответ для query по умножению матрицы на матрицу (чуть-чуть изменена из семинара)"""
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    #  сортируем полученные значения близостей
    sims = enumerate(sims)
    sorted_docs = sorted(sims, key=lambda x: x[1], reverse=True)
    top1_doc = sorted_docs[0][0]
    return answers_train[top1_doc]

Находим топ1 для каждого запроса и меряем качество:

In [141]:
predicted = [search(matrix_docs, q) for q in matrix_queries]
predicted_ner = [search(matrix_docs_ner, q) for q in matrix_queries_ner]

In [142]:
print('Качество поиска в экспериментальном методе перемножения двух матриц: ', accuracy(predicted))
print('Качество поиска в экспериментальном методе перемножения двух матриц с удалением NER', accuracy(predicted_ner))

Качество поиска в экспериментальном методе перемножения двух матриц:  0.44395924308588064
Качество поиска в экспериментальном методе перемножения двух матриц с удалением NER 0.43377001455604075


Этот подход показал себя намного лучше, чем предыдущий!